In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
import classyRF_final as classyRF
import pandas as pd
from astropy.table import Column, Table, vstack

# Creation of parameter sweep (if needed) #

In [2]:
def param_sweep(pathClassy):
    
    df = pd.DataFrame()
    mass1 = np.arange(1, 70.5, 0.5)
    mass2 = np.arange(1, 70.5, 0.5)
    t = Table(
        data=np.vstack(
            (np.repeat(mass1, mass2.size),
                np.tile(mass2, mass1.size))
        ).T, names=('mass1', 'mass2')
    )
    mask = t['mass1'] > t['mass2']
    t = t[mask]

    spins = Table(
        data=np.vstack(
            (np.repeat(np.linspace(0, 1, 2), 2),
                np.tile(np.linspace(0, 1, 2), 2))
        ).T,
        names=('chi1', 'chi2')
    )
    SNR = 10.
    res = list()
    for spin_vals in spins:
        SNR *= np.ones(len(t))
        chi1 = spin_vals['chi1'] * np.ones(len(t))
        chi2 = spin_vals['chi2'] * np.ones(len(t))
        # make predictions and make plots
        param_sweep_features = np.stack(
            [t['mass1'], t['mass2'], chi1, chi2, SNR]
        ).T
    # both KNN and mass_gap are binary classifier with p(false) p(true)
    # P(true) can be accessed with predict_proba(data).T[1]
        res.append(param_sweep_features)
    param_sweep = np.reshape(param_sweep,(550*9591,5))
    pd.DataFrame(param_sweep).to_csv(pathClassy+"sweep_v2.csv")
    
    return 

# Functions for testing dataset and O3/param_sweep events

In [17]:
def matrix_forest(RFdic, ntrees,eos,norm = False):
    
    total_events = len(RFdic['forest'].labels_test)
    
    predictions_trees =np.zeros(shape=(ntrees,len(RFdic['forest'].labels_test)))
    
    nplus_NS = np.zeros(total_events)
    nplus_REM = np.zeros(total_events)
    nplus_no = np.zeros(total_events)
    
    for i in range(0,ntrees):
        predictions_trees[i] = RFdic['forest'].model.estimators_[i].predict(RFdic['forest'].data_test)
    
    for i in range(0,total_events):
        for ll in range(0,len(predictions_trees[:,i])): 
            predictions_trees[ll,i] = int(predictions_trees[ll,i])
        labels = list(predictions_trees[:,i])
        if norm == False:
            nplus_NS[i] = int(labels.count(1))
            nplus_REM[i] = int(labels.count(2))
            nplus_no[i] = int(labels.count(0))
        else:    
            nplus_NS[i] = int(labels.count(1))/ntrees
            nplus_REM[i] = int(labels.count(2))/ntrees
            nplus_no[i] = int(labels.count(0))/ntrees
            

    data = pd.DataFrame({'HasNS_true':RFdic['forest'].label_NS})
    data['HasREM_true'] = RFdic['forest'].label_REM
    data['f0'] = nplus_no
    data['f1'] = nplus_NS
    data['f2'] = nplus_REM
     
    data.to_csv('matrix_RF_O2_EOS-'+eos+'.csv', index = False)
    
    return 

def matrix_forest_O3(RFdic, ntrees,eos,events, events_id = None, o3 = False, norm = False ):
    
    total_events = len(events[:,0])
    
    predictions_trees =np.zeros(shape=(ntrees,len(events[:,0])))
    
    nplus_NS = np.zeros(total_events)
    nplus_REM = np.zeros(total_events)
    nplus_no = np.zeros(total_events)
    
    for i in range(0,ntrees):
        predictions_trees[i] = RFdic['forest'].model.estimators_[i].predict(events[:])
    
    for i in range(0,total_events):
        for ll in range(0,len(predictions_trees[:,i])): 
            predictions_trees[ll,i] = int(predictions_trees[ll,i])
        labels = list(predictions_trees[:,i])
        if norm == False:
            nplus_NS[i] = int(labels.count(1))
            nplus_REM[i] = int(labels.count(2))
            nplus_no[i] = int(labels.count(0))
        else:    
            nplus_NS[i] = int(labels.count(1))/ntrees
            nplus_REM[i] = int(labels.count(2))/ntrees
            nplus_no[i] = int(labels.count(0))/ntrees
        
        if o3 == True:
            print('Event ID: ', events_id[i])
            print(labels)    
            print('Nothing: ',nplus_no[i])
            print('NS: ',nplus_NS[i])
            print('REM: ',nplus_REM[i])
            print('*'*60)
        
    return nplus_NS, nplus_REM, nplus_no

# Functions for checking if the trees are counted properly 

In [15]:
def matrix_forest_test(RFdic, ntrees,eos,nrandom,norm = False):
    
    total_events = len(RFdic['forest'].labels_test)
    
    rnd_ev = random.sample(range(0,total_events), nrandom)
    print('random events: ',rnd_ev)
    
    total_events = len(rnd_ev)
    
    true_labels = RFdic['forest'].labels_test[rnd_ev]
    
    predictions_trees =np.zeros(shape=(ntrees,len(RFdic['forest'].labels_test[rnd_ev])))
    
    nplus_NS = np.zeros(total_events)
    nplus_REM = np.zeros(total_events)
    nplus_no = np.zeros(total_events)
    
    for i in range(0,ntrees):
        predictions_trees[i] = RFdic['forest'].model.estimators_[i].predict(RFdic['forest'].data_test[rnd_ev])
    
    for i in range(0,total_events):
        for ll in range(0,len(predictions_trees[:,i])): 
            predictions_trees[ll,i] = int(predictions_trees[ll,i])
        labels = list(predictions_trees[:,i])
        print(labels)
        if norm == False:
            nplus_NS[i] = int(labels.count(1))
            nplus_REM[i] = int(labels.count(2))
            nplus_no[i] = int(labels.count(0))
        else:    
            nplus_NS[i] = int(labels.count(1))/ntrees
            nplus_REM[i] = int(labels.count(2))/ntrees
            nplus_no[i] = int(labels.count(0))/ntrees
            
        print(true_labels[i])
        print('Nothing: ',nplus_no[i])
        print('NS: ',nplus_NS[i])
        print('REM: ',nplus_REM[i])
        print('*'*60)
        
    return 

def matrix_forest_O3_test(RFdic, ntrees,eos,events,nrandom, events_id = None, o3 = False, norm = False ):
    
    total_events = len(events[:,0])
    
    rnd_ev = random.sample(range(0,total_events), nrandom)
    print('random events: ',rnd_ev)
    
    total_events = len(rnd_ev)

    predictions_trees =np.zeros(shape=(ntrees,len(events[rnd_ev,0])))
    
    nplus_NS = np.zeros(total_events)
    nplus_REM = np.zeros(total_events)
    nplus_no = np.zeros(total_events)
    
    for i in range(0,ntrees):
        predictions_trees[i] = RFdic['forest'].model.estimators_[i].predict(events[rnd_ev])
    
    for i in range(0,total_events):
        for ll in range(0,len(predictions_trees[:,i])): 
            predictions_trees[ll,i] = int(predictions_trees[ll,i])
        labels = list(predictions_trees[:,i])
        if norm == False:
            nplus_NS[i] = int(labels.count(1))
            nplus_REM[i] = int(labels.count(2))
            nplus_no[i] = int(labels.count(0))
        else:    
            nplus_NS[i] = int(labels.count(1))/ntrees
            nplus_REM[i] = int(labels.count(2))/ntrees
            nplus_no[i] = int(labels.count(0))/ntrees
            
        if o3 == True:
            print('Event ID: ', events_id[i])
        
        print(labels)    
        print('Nothing: ',nplus_no[i])
        print('NS: ',nplus_NS[i])
        print('REM: ',nplus_REM[i])
        print('*'*60)
        
    return nplus_NS, nplus_REM, nplus_no


### Definition of paths 

In [5]:
pathClassy = "/Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNN_miq/"
pathData = "/Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNN_miq/input/"
pathForest = "/Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNNvsRF/RF_marina/data/"
EOS = ["APR4_EPP", "BHF_BBB2", "H4", "HQC18", "KDE0V", "KDE0V1", "MPA1", "MS1_PP", "MS1B_PP", "RS", "SK255", "SK272", "SKI2", "SKI3", "SKI4", "SKI5", "SKI6", "SKMP", "SKOP", "SLy", "SLY2", "SLY9", "SLY230A"]
BayesFactor = [1.526, 1.555, 0.056, 1.422, 1.177, 1.283, 0.276, 0.001, 0.009, 0.176, 0.179, 0.159, 0.108, 0.107, 0.33, 0.025, 0.288, 0.29, 0.618, 1.0, 1.028, 0.37, 0.932]

In [6]:
EOSdic = {}
count = 0

## Loading RF trained algorithms

In [7]:
i_m1r=1; i_m2r=2; i_x1r=3; i_x2r=4; i_snr=8;
recovered=[i_m1r, i_m2r, i_x1r, i_x2r, i_snr]
EOSdicRF = {}
count = 0
for eos in EOS:
    RF = classyRF.ClassificationRF()
    RF.loadPickle(pathForest+'random_forest_'+eos)
    RF.load_test_dataset(pathData+eos+'/EMB/', 'test_EMB_ID_LABEL_'+eos+'_s300_f0d7.csv')
    aux = RF.labels_test
    RF.label_REM = aux
    RF.load_test_dataset(pathData+eos+'/NS/', 'test_NS_ID_LABEL_'+eos+'_s300_f0d7.csv')
    RF.label_NS = RF.labels_test
    RF.labels_test = RF.labels_test + aux
    RF.subset_test(recovered)
    new_dic = {}
    new_dic['forest'] = RF
    new_dic['weight'] = BayesFactor[count]
    EOSdicRF[eos] = new_dic
    count = count + 1 

## Computing fractions of the O2 test dataset

In [19]:
for eos in EOS:
    print('Doing EOS: ', eos)
    print('='*60)
    matrix_forest(EOSdicRF[eos],50,eos, norm = True)
    print('#'*60)

Doing EOS:  APR4_EPP
############################################################
Doing EOS:  BHF_BBB2
############################################################
Doing EOS:  H4
############################################################
Doing EOS:  HQC18
############################################################
Doing EOS:  KDE0V
############################################################
Doing EOS:  KDE0V1
############################################################
Doing EOS:  MPA1
############################################################
Doing EOS:  MS1_PP
############################################################
Doing EOS:  MS1B_PP
############################################################
Doing EOS:  RS
############################################################
Doing EOS:  SK255
############################################################
Doing EOS:  SK272
############################################################
Doing EOS:  SKI2
################################

## Take O3 dataset 

In [57]:
df = pd.read_csv(pathClassy+'real_data/real_data.csv')
df.head()

,event_id,grace_id,m1_rec,m2_rec,chi1_rec,chi2_rec,snr,GWTC,pHasNS,pHasREM
0,GW170823,G298936,59.126324,24.816019,-0.298205,0.898520,11.2960,1,0.000,0.0
1,GW170817,G298048,1.527005,1.242296,-0.015902,-0.035747,14.4500,1,1.000,1.0
2,GW170814,G297595,29.478622,24.901943,-0.568798,0.130793,16.1496,1,0.000,0.0
3,GW170809,G296853,43.061466,30.084999,-0.120968,0.846714,11.2619,1,0.002,0.0
4,GW190408,G329243,32.132198,23.224018,0.279017,-0.647443,13.9286,2,0.000,0.0


In [65]:
columns = df[['m1_rec', 'm2_rec', 'chi1_rec', 'chi2_rec', 'snr']]

# Convert the DataFrame to a Numpy array
events = columns.values
nplus_NS = []
nplus_REM = []
nplus_NO = []
events_id = []
for i in range(0,len(events[:,0])):
    events_id.append(df['event_id'][i])

## Compute trees for O3 real data

In [66]:
for eos in EOS:
    print('Doing EOS: ', eos)
    print('='*60)
    npns,nprem,npno = matrix_forest_O3(EOSdicRF[eos],50,eos,events,events_id, o3 = True)
    nplus_NS.append(npns)
    nplus_REM.append(nprem)
    nplus_NO.append(npno)
    print('#'*60)

Doing EOS:  APR4_EPP
Event ID:  GW170823
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Nothing:  50.0
NS:  0.0
REM:  0.0
************************************************************
Event ID:  GW170817
[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]
Nothing:  0.0
NS:  0.0
REM:  50.0
************************************************************
Event ID:  GW170814
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

NS:  0.0
REM:  50.0
************************************************************
Event ID:  GW170814
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Nothing:  50.0
NS:  0.0
REM:  0.0
************************************************************
Event ID:  GW170809
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Nothing:  50.0
NS:  0.0
REM:  0.0
************************************************************
Event ID:  GW190408
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

REM:  0.0
************************************************************
Event ID:  GW190413_134308
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Nothing:  50.0
NS:  0.0
REM:  0.0
************************************************************
Event ID:  GW190421
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Nothing:  50.0
NS:  0.0
REM:  0.0
************************************************************
Event ID:  GW190425
[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0,

************************************************************
Event ID:  GW191215
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Nothing:  50.0
NS:  0.0
REM:  0.0
************************************************************
Event ID:  GW191216
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Nothing:  49.0
NS:  1.0
REM:  0.0
************************************************************
Event ID:  GW191222
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

Event ID:  GW170823
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Nothing:  50.0
NS:  0.0
REM:  0.0
************************************************************
Event ID:  GW170817
[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]
Nothing:  0.0
NS:  0.0
REM:  50.0
************************************************************
Event ID:  GW170814
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

Event ID:  GW170823
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Nothing:  50.0
NS:  0.0
REM:  0.0
************************************************************
Event ID:  GW170817
[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]
Nothing:  0.0
NS:  0.0
REM:  50.0
************************************************************
Event ID:  GW170814
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

Event ID:  GW170823
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Nothing:  50.0
NS:  0.0
REM:  0.0
************************************************************
Event ID:  GW170817
[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]
Nothing:  0.0
NS:  0.0
REM:  50.0
************************************************************
Event ID:  GW170814
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

Event ID:  GW191216
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Nothing:  50.0
NS:  0.0
REM:  0.0
************************************************************
Event ID:  GW191222
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Nothing:  50.0
NS:  0.0
REM:  0.0
************************************************************
Event ID:  GW200112
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

Event ID:  GW170823
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Nothing:  50.0
NS:  0.0
REM:  0.0
************************************************************
Event ID:  GW170817
[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]
Nothing:  0.0
NS:  0.0
REM:  50.0
************************************************************
Event ID:  GW170814
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

Event ID:  GW190924
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Nothing:  48.0
NS:  2.0
REM:  0.0
************************************************************
Event ID:  GW190930
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Nothing:  50.0
NS:  0.0
REM:  0.0
************************************************************
Event ID:  GW191109
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

## Save fractions/number of trees of O3 data

In [68]:
data = pd.DataFrame({'event_ID':event_ids})

for i,eos in enumerate(EOS):
    data['f0_'+eos] = nplus_NO[i]
    data['f1_'+eos] = nplus_NS[i]
    data['f2_'+eos] = nplus_REM[i]
    
    
data.to_csv('matrix_RF_O3_all_EOS.csv', index = False)   

## Read parameter sweep file 

In [9]:
df = pd.read_csv(pathClassy+'sweeps/sweep_v2.csv')

In [10]:
events = df.values
events = events[:,1:]
print(events)

[[ 1.39698492  1.          0.          0.         10.        ]
 [ 1.79396985  1.          0.          0.         10.        ]
 [ 1.79396985  1.39698492  0.          0.         10.        ]
 ...
 [80.         78.80904523  1.          1.         10.        ]
 [80.         79.20603015  1.          1.         10.        ]
 [80.         79.60301508  1.          1.         10.        ]]


In [16]:
nplus_NS = []
nplus_REM = []
nplus_NO = []
for eos in EOS:
    print('Doing EOS: ', eos)
    print('='*60)
    npns,nprem,npno = matrix_forest_O3(EOSdicRF[eos],50,eos,events, o3 = False)
    nplus_NS.append(npns)
    nplus_REM.append(nprem)
    nplus_NO.append(npno)
    print('#'*60)

Doing EOS:  APR4_EPP
############################################################
Doing EOS:  BHF_BBB2
############################################################
Doing EOS:  H4
############################################################
Doing EOS:  HQC18
############################################################
Doing EOS:  KDE0V
############################################################
Doing EOS:  KDE0V1
############################################################
Doing EOS:  MPA1
############################################################
Doing EOS:  MS1_PP
############################################################
Doing EOS:  MS1B_PP
############################################################
Doing EOS:  RS
############################################################
Doing EOS:  SK255
############################################################
Doing EOS:  SK272
############################################################
Doing EOS:  SKI2
################################

## Saving results into file

In [63]:
data = pd.DataFrame({'event_ID':np.arange(0,len(events[:,0]))})

for i,eos in enumerate(EOS):
    data['f0_'+eos] = nplus_NO[i]
    data['f1_'+eos] = nplus_NS[i]
    data['f2_'+eos] = nplus_REM[i]
    
    
data.to_csv('matrix_RF_param_sweep_v2.csv', index = False)   

## Weighting the fractions (deprecated...)

In [24]:
nplusNSw = np.zeros(len(df));  nplusREMw = np.zeros(len(df)); nplusNOw = np.zeros(len(df));
for i,eos in enumerate(EOS):
    nplusNSw += nplus_NS[i] * EOSdicRF[eos]['weight']
    nplusREMw += nplus_REM[i]  * EOSdicRF[eos]['weight']
    nplusNOw += nplus_NO[i]  * EOSdicRF[eos]['weight']
total = sum(BayesFactor)

[954, 658, 814, 117, 56, 495, 499, 548, 72, 915]

In [ ]:
nplusNSw = nplusNSw/total
nplusREMw = nplusREMw/total
nplusNOw = nplusNOw/total

In [ ]:
with open('matrix_events_RF_O3_all_EOS.txt','w+') as f:

        f.write('# event_id \t f_0 \t f_1 \t f_2 \n')

        for i in range(0,len(events[:,0])):
            f.write('%s \t\t %.4f \t %.4f \t %.4f \n'%(event_ids[i],nplusNOw[i],nplusNSw[i], nplusREMw[i]))
    